<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/tf/b4_public_datasets_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using public datasets with TF Datasets

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers

tfds.__version__

'4.0.1'

In [2]:
mnist_data = tfds.load('fashion_mnist')
type(mnist_data), mnist_data

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteNVCS1X/fashion_mnist-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteNVCS1X/fashion_mnist-test.tfrecord


Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


(dict,
 {'test': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
  'train': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>})

In [3]:
for item in mnist_data:
  print(type(item), item)

<class 'str'> test
<class 'str'> train


If you want to load these splits into a dataset containing the actual data, you can simply specify the split you want in the tfds.load command, like this:

In [4]:
mnist_train = tfds.load(name='fashion_mnist', split='train')
assert isinstance(mnist_train, tf.data.Dataset)
type(mnist_train)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In this instance, we we a `PrefetchDataset` object, which we can iterate through to inspect the data. One nice feature is that we can apply `take(1)` and get the first record.

In [5]:
item = next(iter(mnist_train.take(1)))
print(type(item))
print(item.keys())

<class 'dict'>
dict_keys(['image', 'label'])


In [6]:
image = item['image']
print(type(image))
print(image.shape)
print(image[0:0])

<class 'tensorflow.python.framework.ops.EagerTensor'>
(28, 28, 1)
tf.Tensor([], shape=(0, 28, 1), dtype=uint8)


In [7]:
label = item['label']
print(type(label))
print(label)

<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(2, shape=(), dtype=int64)


In [8]:
mnist_test, info = tfds.load(name='fashion_mnist', with_info='true')
info

tfds.core.DatasetInfo(
    name='fashion_mnist',
    version=3.0.1,
    description='Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.',
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author    = {Han Xiao and
                   Kashif Rasul and
                   Roland Vollgraf},
      title     = {Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning
                   Algorithms},
      journal   = {CoRR},
      volume

## Using TFDS with Keras Model

In [9]:
mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(type(train_images))

4423680/4422102 [==============================] - 0s 0us/step
<class 'numpy.ndarray'>


When using TFDS the code is very similar, but with some minor changes. The Keras datasets gave us `ndarray` that worked natively in `model.fit`. However, with TFDS we will need to do a little conversion work.

In [10]:
(train_images, train_labels), (test_images, test_labels) = \
  tfds.as_numpy(
      tfds.load('fashion_mnist',
                split=['train', 'test'],
                batch_size=-1,
                as_supervised=True))
print(type(train_images))

<class 'numpy.ndarray'>


In [11]:
# we need to rescale our images before feeding them into the network
# train_images = train_images * 1.0/255.0
# test_images = test_images * 1.0/255.0
# skipping this rescaling step in favor of adding rescaling directly
# into the model pipeline(see layers...Rescaling) 

model = tf.keras.models.Sequential([
  layers.experimental.preprocessing.Rescaling(1.0/255.0),
  layers.Flatten(input_shape=(28, 28, 1)),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(10, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

model.fit(
    train_images,
    train_labels,
    epochs=5
)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5273 - accuracy: 0.8144
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3990 - accuracy: 0.8569
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3659 - accuracy: 0.8662
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3425 - accuracy: 0.8732
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3302 - accuracy: 0.8784


The data is batched and shuffled to make training more effective?

## Human-or-Horses Model

In [14]:
data = tfds.load('horses_or_humans', split='train', as_supervised=True)
val_data = tfds.load('horses_or_humans', split='test', as_supervised=True)
 
train_batches = data.shuffle(100).batch(10)
validation_batches = val_data.batch(32)

model = tf.keras.models.Sequential([
    layers.experimental.preprocessing.Rescaling(1.0/255.0),
    layers.Conv2D(16, (3,3), activation='relu', 
                           input_shape=(300, 300, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='Adam', loss='binary_crossentropy',
metrics=['accuracy'])

history = model.fit(
    train_batches, 
    epochs=10,
    validation_data=validation_batches
)

Epoch 1/10
103/103 [==============================] - 5s 48ms/step - loss: 0.2991 - accuracy: 0.8500 - val_loss: 0.9548 - val_accuracy: 0.8320
Epoch 2/10
103/103 [==============================] - 5s 45ms/step - loss: 0.1453 - accuracy: 0.9523 - val_loss: 1.8190 - val_accuracy: 0.8164
Epoch 3/10
103/103 [==============================] - 5s 45ms/step - loss: 0.0717 - accuracy: 0.9786 - val_loss: 1.4623 - val_accuracy: 0.8984
Epoch 4/10
103/103 [==============================] - 5s 46ms/step - loss: 0.0788 - accuracy: 0.9698 - val_loss: 4.2458 - val_accuracy: 0.7891
Epoch 5/10
103/103 [==============================] - 5s 45ms/step - loss: 0.0456 - accuracy: 0.9815 - val_loss: 2.3205 - val_accuracy: 0.8047
Epoch 6/10
103/103 [==============================] - 5s 45ms/step - loss: 0.0361 - accuracy: 0.9883 - val_loss: 3.2655 - val_accuracy: 0.8086
Epoch 7/10
103/103 [==============================] - 5s 44ms/step - loss: 0.0394 - accuracy: 0.9854 - val_loss: 1.1305 - val_accuracy: 0.8555